In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#plt.rcParams['text.usetex'] = True

In [ ]:
# 두꺼운펜변환 함수 (사각펜)
def thick_pen_transform(X, taus, gamma=0.001):
    n = len(X)
    result = {}
    
    for tau in taus:
        U_tau = np.zeros(n)
        L_tau = np.zeros(n)
        
        for t in range(n - tau):
            U_tau[t] = np.max(X[t:t+tau+1]) + gamma * (tau / 2)
            L_tau[t] = np.min(X[t:t+tau+1]) - gamma * (tau / 2)
        
        # 경계 케이스 처리
        for t in range(n - tau, n):
            U_tau[t] = np.max(X[t:n]) + gamma * (tau / 2)
            L_tau[t] = np.min(X[t:n]) - gamma * (tau / 2)
        
        result[tau] = (L_tau, U_tau)
    
    return result

# 두꺼운펜 변환 상관계수 계산 함수
def rho_tau(X, Y, taus):
    rho_values = {}
    
    X_result = thick_pen_transform(X, taus)
    Y_result = thick_pen_transform(Y, taus)
    
    for tau in taus:
        rho_tau_t = np.zeros(len(X))
        
        L_tau_X, U_tau_X = X_result[tau]
        L_tau_Y, U_tau_Y = Y_result[tau]
        
        for t in range(len(X)):
            min_U = min(U_tau_X[t], U_tau_Y[t])
            max_L = max(L_tau_X[t], L_tau_Y[t])
            max_U = max(U_tau_X[t], U_tau_Y[t])
            min_L = min(L_tau_X[t], L_tau_Y[t])
            
            rho_tau_t[t] = (min_U - max_L) / (max_U - min_L)
        
        rho_values[tau] = rho_tau_t
    
    return rho_values

def plot_correlations(indices, start_date, end_date, taus):
    # 데이터 다운로드 및 전처리
    data_dict = {}
    min_length = float('inf')

    for name, ticker in indices.items():
        data = yf.download(ticker, start=start_date, end=end_date)
        close_prices = data['Close'].values
        dates = data.index

        # 로그 변환
        log_prices = np.log(close_prices)

        # 차분
        differenced_log_prices = np.diff(log_prices)

        # 표준화
        mean_diff = np.mean(differenced_log_prices)
        std_diff = np.std(differenced_log_prices)
        standardized_diff = (differenced_log_prices - mean_diff) / std_diff

        data_dict[name] = standardized_diff
        if len(standardized_diff) < min_length:
            min_length = len(standardized_diff)

    # 데이터 길이를 최소 길이로 맞춤
    for name in data_dict:
        data_dict[name] = data_dict[name][:min_length]

    all_rho_values = []

    # 데이터 딕셔너리에서 처음 7개의 항목만 선택
    selected_indices = list(data_dict.keys())

    for i, name1 in enumerate(selected_indices):
        for j, name2 in enumerate(selected_indices):
            if name1 != name2:  # 자기 자신과의 상관계수는 계산하지 않음
                rho_values = rho_tau(data_dict[name1], data_dict[name2], taus)
                
                rho_matrix = np.zeros((len(taus), min_length))

                for k, tau in enumerate(taus):
                    rho_matrix[k, :] = rho_values[tau]
                
                all_rho_values.append(rho_matrix)
    # 전체 데이터의 상위 x% 임계값 계산
    all_rho_values = np.concatenate(all_rho_values)
    global_threshold = np.percentile(all_rho_values, 0)

    covid_start = pd.Timestamp('2020-03-11')
    covid_end = pd.Timestamp('2021-12-31')

    # 날짜 데이터 추출 (첫 번째 지수의 날짜를 사용)
    dates = yf.download(indices[selected_indices[0]], start=start_date, end=end_date).index

    # 이벤트 기간을 인덱스로 변환
    covid_start_idx = np.where(dates >= covid_start)[0][0]
    covid_end_idx = np.where(dates <= covid_end)[0][-1]

    # X축 레이블 설정을 위해 연도로 변환
    years = dates.year

    n = len(selected_indices)
    fig, axes = plt.subplots(nrows=n, ncols=n, figsize=(25, 25))
    for i, name1 in enumerate(selected_indices):
        for j, name2 in enumerate(selected_indices):
            ax = axes[i, j]
            
            if name1 == name2:  # 자기 자신과의 상관계수 위치에 'X' 표시
                ax.text(0.5, 0.5, 'X', fontsize=40, ha='center', va='center')
                ax.axis('off')
            else:
                rho_values = rho_tau(data_dict[name1], data_dict[name2], taus)
                rho_matrix = np.zeros((len(taus), min_length))

                for k, tau in enumerate(taus):
                    rho_matrix[k, :] = rho_values[tau]

                # 전역 임계값을 사용하여 마스크 생성
                mask = rho_matrix < global_threshold

                # 마스크를 적용한 데이터 생성
                masked_data = np.ma.array(rho_matrix, mask=mask)

                im = ax.imshow(masked_data, cmap='magma', origin='lower', extent=[0, min_length, min(taus), max(taus)], alpha=0.5)
                ax.set_aspect('auto')
                
                # 코로나19 팬데믹 기간 표시
                ax.axvline(x=covid_start_idx, color='r', linestyle='--', linewidth=0.5); # ax.axvline(x=covid_end_idx, color='r', linestyle='--', linewidth=0.5)
                
                ax.set_title(f'{name1} vs {name2}', fontsize=8)
                ax.set_xlabel('Time (Years)', fontsize=6)
                ax.set_ylabel('Tau', fontsize=6)
                ax.tick_params(axis='both', which='major', labelsize=6)
                
                # X축에 연도 레이블 추가
                ax.set_xticks(np.linspace(0, min_length, len(set(years))))
                ax.set_xticklabels(sorted(set(years)))

            if i == 6 and j == 6:  # 마지막 셀에만 컬러바 추가
                plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.2, aspect=30, shrink=0.8)

    # 범례 추가
    fig.legend(['COVID-19'], loc='upper right', bbox_to_anchor=(0.99, 0.99), fontsize=8)

    plt.show()
    plt.tight_layout()  

def plot_correlations2(indices, start_date, end_date, taus, sigma=1):
    # 데이터 다운로드 및 전처리
    data_dict = {}
    min_length = float('inf')

    for name, ticker in indices.items():
        data = yf.download(ticker, start=start_date, end=end_date)
        close_prices = data['Close'].values
        dates = data.index

        # 로그 변환
        log_prices = np.log(close_prices)

        # 차분
        differenced_log_prices = np.diff(log_prices)

        # 표준화
        mean_diff = np.mean(differenced_log_prices)
        std_diff = np.std(differenced_log_prices)
        standardized_diff = (differenced_log_prices - mean_diff) / std_diff

        data_dict[name] = standardized_diff
        if len(standardized_diff) < min_length:
            min_length = len(standardized_diff)

    # 데이터 길이를 최소 길이로 맞춤
    for name in data_dict:
        data_dict[name] = data_dict[name][:min_length]

    all_rho_values = []

    # 데이터 딕셔너리에서 처음 7개의 항목만 선택
    selected_indices = list(data_dict.keys())

    for i, name1 in enumerate(selected_indices):
        for j, name2 in enumerate(selected_indices):
            if name1 != name2:  # 자기 자신과의 상관계수는 계산하지 않음
                rho_values = rho_tau(data_dict[name1], data_dict[name2], taus)
                
                rho_matrix = np.zeros((len(taus), min_length))

                for k, tau in enumerate(taus):
                    rho_matrix[k, :] = rho_values[tau]
                
                all_rho_values.append(rho_matrix)
    # 전체 데이터의 상위 x% 임계값 계산
    all_rho_values = np.concatenate(all_rho_values)
    global_threshold = np.percentile(all_rho_values, 0)

    # Min-Max 스케일링
    min_val = np.min(all_rho_values)
    max_val = np.max(all_rho_values)

    covid_start = pd.Timestamp('2020-03-11')
    #covid_end = pd.Timestamp('2021-12-31')

    # 날짜 데이터 추출 (첫 번째 지수의 날짜를 사용)
    dates = yf.download(indices[selected_indices[0]], start=start_date, end=end_date).index

    # 이벤트 기간을 인덱스로 변환
    covid_start_idx = np.where(dates >= covid_start)[0][0]
    #covid_end_idx = np.where(dates <= covid_end)[0][-1]

    # X축 레이블 설정을 위해 연도로 변환
    years = dates.year

    n = len(selected_indices)
    fig, axes = plt.subplots(nrows=n, ncols=n, figsize=(25, 25))
    for i, name1 in enumerate(selected_indices):
        for j, name2 in enumerate(selected_indices):
            ax = axes[i, j]
            
            if name1 == name2:  # 자기 자신과의 상관계수 위치에 'X' 표시
                ax.text(0.5, 0.5, 'X', fontsize=40, ha='center', va='center')
                ax.axis('off')
            else:
                rho_values = rho_tau(data_dict[name1], data_dict[name2], taus)
                rho_matrix = np.zeros((len(taus), min_length))

                for k, tau in enumerate(taus):
                    rho_matrix[k, :] = rho_values[tau]

                # 시간별로 스무딩 적용
                smoothed_rho_matrix = gaussian_filter(rho_matrix, sigma=sigma)

                # 전역 임계값을 사용하여 마스크 생성
                mask = smoothed_rho_matrix < global_threshold

                # 마스크를 적용한 데이터 생성
                masked_data = np.ma.array(smoothed_rho_matrix, mask=mask)

                im = ax.imshow(masked_data, cmap='magma', origin='lower', extent=[0, min_length, min(taus), max(taus)], alpha=0.8)
                ax.set_aspect('auto')
                
                # 코로나19 팬데믹 기간 표시
                ax.axvline(x=covid_start_idx, color='lime', linestyle='--', linewidth=1.5)#; ax.axvline(x=covid_end_idx, color='lime', linestyle='--', linewidth=1.5)
                
                ax.set_title(f'{name1} vs {name2}', fontsize=8)
                ax.set_xlabel('Time (Years)', fontsize=6)
                ax.set_ylabel('Tau', fontsize=6)
                ax.tick_params(axis='both', which='major', labelsize=6)
                
                # X축에 연도 레이블 추가
                ax.set_xticks(np.linspace(0, min_length, len(set(years))))
                ax.set_xticklabels(sorted(set(years)))

            if i == 6 and j == 6:  # 마지막 셀에만 컬러바 추가
                plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.2, aspect=30, shrink=0.8)

    # 범례 추가
    fig.legend(['COVID-19'], loc='upper right', bbox_to_anchor=(0.99, 0.99), fontsize=8)

    plt.show()  
    plt.tight_layout()  

In [50]:
# 예시 사용법
indices = {
    'Apple Inc.': 'AAPL',
    'Microsoft Corporation': 'MSFT',
    'Nvidia Corporation': 'NVDA',
    'Alphabet Inc. (Google)': 'GOOGL',
    'Amazon.com Inc.': 'AMZN',
    'Tesla Inc.': 'TSLA',
    'Meta Platforms Inc.': 'META'
}
start_date = '2019-06-25'
end_date = '2024-06-25'
taus = list(range(5, 300, 5))

plot_correlations2(indices, start_date, end_date, taus)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
